In [2]:
import numpy as np
import scipy as sp
import pandas as pd
import random
import matplotlib.pyplot as plt
import  networkx as nx
import tensorflow as tf

%matplotlib inline

In [7]:
#function for laplacian

#get diagonal matrix from adjacency matrix
x = np.random.rand(2,2)

x_tensor = tf.cast(x, tf.float32)

degree_m_inverse = tf.diag(tf.inv(tf.reduce_sum(x_tensor,0))) #takes the degree of each vertex and makes diagonal matrix out of it

laplacian = tf.matmul(degree_m_inverse, x_tensor)


with tf.Session() as sess:
    a, b = sess.run([laplacian, degree_m_inverse])

#double check we do the same thing in np.


degree_m = np.diag(np.reciprocal(np.ndarray.sum(np.array(x, dtype = np.float32), 0)))
laplacian_np = np.dot(degree_m, x)

print a, laplacian_np

[[ 0.56161034  0.51751173]
 [ 0.4695904   0.44565633]] [[ 0.56161033  0.51751172]
 [ 0.46959038  0.44565635]]


In [8]:
def joint_permutation(A):
    #takes adjacency matrix and relabels, gives out permutated adjacency matrix of same relationship
    random_shuffle = np.random.permutation(len(A))

    A_shuffle = A[random_shuffle]
    A_shuffle = np.transpose(A_shuffle)
    A_shuffle = A_shuffle[random_shuffle]

    return A_shuffle, random_shuffle

def balanced_stochastic_blockmodel(communities=2, groupsize=3, p_in=1.0, p_out=0.0):
    #gives dense adjacency matrix representaiton of randomly generated SBM with balanced community size

    G = nx.planted_partition_graph(l=communities, k=groupsize, p_in=p_in, p_out =p_out)
    A = nx.adjacency_matrix(G).todense()
    
    return A

In [10]:
A = balanced_stochastic_blockmodel(communities=2, groupsize=3, p_in=1.0, p_out=0.0)
B = joint_permutation(A)
print A, B

x = A
dim_graph = len(x)
k = 2

x_tensor = tf.cast(x, tf.float32)
 #takes the degree of each vertex and makes diagonal matrix out of it
laplacian = tf.matmul(tf.diag(tf.inv(tf.reduce_sum(x_tensor,0))),
                      x_tensor)
#the laplacian is symmetric, we wish to get the k largest eigenvalues

eigenval, eigenvec = tf.self_adjoint_eig(laplacian) #seems to be sorted for me
Y = tf.slice(eigenvec, [0, dim_graph-k], [dim_graph, k]) #pick the top k eigenvectors


#now we do K-means clustering on the rows of Y, which are the top k eignvectors of the laplacian above, or the bottom k of the normalized laplacian

#find k random centroides

centroides = tf.Variable(tf.slice(tf.random_shuffle(Y),[0,0],[k,-1]))

expanded_Y = tf.expand_dims(Y, 0)
expanded_centroides = tf.expand_dims(centroides, 1)

diff = tf.sub(expanded_Y, expanded_centroides) #will get difference between eacnh centroide and all of thw points

sqr = tf.square(diff) #sqr diff

distances = tf.reduce_sum(sqr, 2)
assignments = tf.argmin(distances, 0) #these are the clustering assignments based on current centroides

means = tf.concat(0, 
                  [tf.reduce_mean(
            tf.gather(
                Y, tf.reshape(
                    tf.where( 
                        tf.equal(assignments, c)),[1,-1])),
            reduction_indices=[1]) for c in xrange(k)])

#these new means, calculated by group, will be the new centroides
update_centroides = tf.assign(centroides, means)

init = tf.initialize_all_variables()


with tf.Session() as sess:
    sess.run(init)
    for step in xrange(100):
        _, centroid_values, assigment_values = sess.run([centroides, update_centroides, assignments])
    print sess.run([centroides, update_centroides, assignments])


[[0 1 1 0 0 0]
 [1 0 1 0 0 0]
 [1 1 0 0 0 0]
 [0 0 0 0 1 1]
 [0 0 0 1 0 1]
 [0 0 0 1 1 0]] (matrix([[0, 0, 1, 1, 0, 0],
        [0, 0, 0, 0, 1, 1],
        [1, 0, 0, 1, 0, 0],
        [1, 0, 1, 0, 0, 0],
        [0, 1, 0, 0, 0, 1],
        [0, 1, 0, 0, 1, 0]]), array([3, 2, 4, 5, 0, 1]))
[array([[ 0.        ,  0.57735026],
       [ 0.57735026,  0.        ]], dtype=float32), array([[ 0.        ,  0.57735026],
       [ 0.57735026,  0.        ]], dtype=float32), array([0, 0, 0, 1, 1, 1])]


In [11]:
communities = 2
group_size = 9
A = balanced_stochastic_blockmodel(communities=communities, groupsize=group_size, p_in=0.8, p_out=0.5)
x = A
x

matrix([[0, 1, 0, 1, 0, 0, 1, 1, 1, 0, 0, 0, 1, 0, 1, 1, 1, 1],
        [1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0, 0, 0],
        [0, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1],
        [1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1],
        [0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1],
        [0, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1],
        [1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0],
        [1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1],
        [1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0],
        [0, 1, 1, 1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 1, 1, 1, 0, 0],
        [0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1, 1, 1, 1, 1, 0, 0],
        [0, 1, 1, 0, 1, 1, 0, 0, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1],
        [1, 0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 0, 0, 1, 1],
        [0, 1, 1, 0, 0, 0, 0, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 0],
        [1, 0, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 0, 1, 0, 1],
        [1, 0, 0, 0, 1, 1, 0, 0, 0, 1, 1

In [12]:
dim_graph = communities*group_size

x = A#tf.placeholder(tf.float32, shape=[dim_graph, dim_graph])

x_tensor = tf.cast(x, tf.float32)


#Laplcian Branch
laplacian = tf.matmul(tf.diag(tf.inv(tf.reduce_sum(x_tensor,0))),x_tensor)
eigenval, eigenvec = tf.self_adjoint_eig(laplacian)
Y = tf.slice(eigenvec, [0, dim_graph-communities], [dim_graph, communities])

#kmeans

centroides = tf.Variable(tf.slice(tf.random_shuffle(Y),[0,0],[communities,-1]))

expanded_Y = tf.expand_dims(Y, 0)
expanded_centroides = tf.expand_dims(centroides, 1)

assignments = tf.argmin(tf.reduce_sum(tf.square(tf.sub(expanded_Y, expanded_centroides)), 2), 0) #these are the clustering assignments based on current centroides
means = tf.concat(0, [tf.reduce_mean(tf.gather(Y, tf.reshape(tf.where( tf.equal(assignments, c)),[1,-1])),
                                     reduction_indices=[1]) for c in xrange(communities)])

update_centroides = tf.assign(centroides, means)


#Adjacnecy Branch

Adj_eigenval, Adj_eigenvec = tf.self_adjoint_eig(x_tensor)
Y_adj = tf.slice(Adj_eigenvec, [0, dim_graph-communities], [dim_graph, communities])

#kmeans 

centroides_adj = tf.Variable(tf.slice(tf.random_shuffle(Y_adj),[0,0],[communities,-1]))

expanded_Y_adj = tf.expand_dims(Y_adj, 0)
expanded_centroides_adj = tf.expand_dims(centroides_adj, 1)

assignments_adj = tf.argmin(tf.reduce_sum(tf.square(tf.sub(expanded_Y_adj, expanded_centroides_adj)), 2), 0) #these are the clustering assignments based on current centroides
means_adj = tf.concat(0, [tf.reduce_mean(tf.gather(Y_adj,
                                                   tf.reshape(tf.where( tf.equal(assignments_adj, c)),[1,-1])),
                                         reduction_indices=[1]) for c in xrange(communities)])

update_centroides_adj = tf.assign(centroides_adj, means_adj)

#optimization

true_assignment_a = tf.concat(0, [tf.zeros([group_size], dtype=tf.float32), tf.ones([group_size], dtype=tf.float32)])
true_assignment_b = tf.concat(0, [tf.ones([group_size], dtype=tf.float32), tf.zeros([group_size], dtype=tf.float32)])         

laplace_assignment = tf.cast(assignments, dtype = tf.float32)
adj_assignment = tf.cast(assignments_adj, dtype = tf.float32)

loss_laplacian = tf.minimum(tf.reduce_sum(tf.square(tf.sub(true_assignment_a, laplace_assignment))), 
                            tf.reduce_sum(tf.square(tf.sub(true_assignment_b, laplace_assignment))))

loss_adj = tf.minimum(tf.reduce_sum(tf.square(tf.sub(true_assignment_a, adj_assignment))),
                      tf.reduce_sum(tf.square(tf.sub(true_assignment_b, adj_assignment))))

error_laplacian = tf.div(loss_laplacian, dim_graph)
error_adj = tf.div(loss_adj, dim_graph)


init = tf.initialize_all_variables()

with tf.Session() as sess:
    sess.run(init)
    for step in xrange(100):
        sess.run([centroides, update_centroides, assignments,
                  centroides_adj, update_centroides_adj, assignments_adj])
    print sess.run([assignments, assignments_adj, error_laplacian, error_adj])



[array([1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0]), array([1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0]), 0.055555556, 0.0]
